## Antes de começar rodar o script:
##### Notebook - Carga Dados Compras

## Lendo a primeira versão via Python

In [0]:
%python
df = spark.read \
.format("delta") \
.option("versionAsOf", "1") \
.load("/user/hive/warehouse/compras")
df.show()

+---+----------+--------+---------------+----+-----+
| id|date_order|customer|        product|unit|price|
+---+----------+--------+---------------+----+-----+
|  2|2021-03-22|     Ana|AR-CONDICIONADO|   6|121.6|
|  5|2021-05-23|  Tereza|       FRIGOBAR|   3|412.0|
|  7|2021-07-25|   Sofia|        CADEIRA|   1|342.3|
|  3|2021-04-21|   Sofia|        FREEZER|   7|415.4|
|  6|2021-06-25|   Carla|           MESA|   1|124.0|
|  4|2021-04-23|  Sandra|             TV|   8|313.0|
|  1|2021-01-23|  Carlos|             TV|   5|238.0|
+---+----------+--------+---------------+----+-----+



## Contando a quantidade de registros na terceira versão via SQL

In [0]:
%sql
SELECT count(*) FROM compras VERSION AS OF 3

count(1)
6


## Contando a quantidade de registros na terceira versão via SQL - Outra forma de realizar a tarefa

In [0]:
%sql
SELECT count(*) FROM compras@v3

count(1)
6


## Contando a quantidade de registros na terceira versão via SQL - Outra forma de realizar a tarefa

In [0]:
%sql
SELECT * FROM delta.`/user/hive/warehouse/compras@v3`

id,date_order,customer,product,unit,price
2,2021-03-22,Ana,AR-CONDICIONADO,6,121.6
4,2021-04-23,Sandra,Geladeira,8,313.0
5,2021-05-23,Tereza,FRIGOBAR,3,412.0
7,2021-07-25,Sofia,CADEIRA,1,342.3
3,2021-04-21,Sofia,FREEZER,7,415.4
6,2021-06-25,Carla,MESA,1,124.0


## Descrevendo o histórico dos dados para verificar a quantidade de versões

In [0]:
%sql
DESCRIBE HISTORY '/user/hive/warehouse/compras'

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
3,2023-07-25T20:18:36.000+0000,3749819615057738,andersonfaro@gmail.com,DELETE,"Map(predicate -> [""(cast(id#20454 as int) = 1)""])",null,List(851213765213230),0725-180139-s9tz1nlq,2,WriteSerializable,false,"Map(numRemovedFiles -> 1, numCopiedRows -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 1774, numDeletedRows -> 1, scanTimeMs -> 1211, numAddedFiles -> 0, rewriteTimeMs -> 563)",null,Databricks-Runtime/10.4.x-scala2.12
2,2023-07-25T20:18:28.000+0000,3749819615057738,andersonfaro@gmail.com,UPDATE,"Map(predicate -> [""(cast(id#19769 as int) = 4)""])",null,List(851213765213230),0725-180139-s9tz1nlq,1,WriteSerializable,false,"Map(numRemovedFiles -> 1, numCopiedRows -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 2955, scanTimeMs -> 32, numAddedFiles -> 1, numUpdatedRows -> 1, rewriteTimeMs -> 2923)",null,Databricks-Runtime/10.4.x-scala2.12
1,2023-07-25T20:18:21.000+0000,3749819615057738,andersonfaro@gmail.com,MERGE,"Map(predicate -> [""(cast(id#18912 as bigint) = id#18330L)""], matchedPredicates -> [{""actionType"":""update""}], notMatchedPredicates -> [{""actionType"":""insert""}])",null,List(851213765213230),0725-180139-s9tz1nlq,0,WriteSerializable,false,"Map(numTargetRowsCopied -> 0, numTargetRowsDeleted -> 0, numTargetFilesAdded -> 7, executionTimeMs -> 3126, numTargetRowsInserted -> 7, scanTimeMs -> 1066, numTargetRowsUpdated -> 0, numOutputRows -> 7, numTargetChangeFilesAdded -> 0, numSourceRows -> 7, numTargetFilesRemoved -> 0, rewriteTimeMs -> 1917)",null,Databricks-Runtime/10.4.x-scala2.12
0,2023-07-25T20:18:14.000+0000,3749819615057738,andersonfaro@gmail.com,CREATE OR REPLACE TABLE,"Map(isManaged -> true, description -> null, partitionBy -> [""date_order""], properties -> {})",null,List(851213765213230),0725-180139-s9tz1nlq,null,WriteSerializable,true,Map(),null,Databricks-Runtime/10.4.x-scala2.12


## Vamos reinserir o registro com ID=1 que eliminamos, uma forma de realizar o Delta Time Travel

In [0]:
%sql
INSERT INTO compras
SELECT * FROM compras VERSION AS OF 1
WHERE Id = 1

num_affected_rows,num_inserted_rows
1,1


## Exibindo os dados atualizados, após retorno da versão 1, ou seja o registro deletado foi restaurado

In [0]:
%sql
SELECT * FROM compras

id,date_order,customer,product,unit,price
2,2021-03-22,Ana,AR-CONDICIONADO,6,121.6
4,2021-04-23,Sandra,Geladeira,8,313.0
5,2021-05-23,Tereza,FRIGOBAR,3,412.0
7,2021-07-25,Sofia,CADEIRA,1,342.3
3,2021-04-21,Sofia,FREEZER,7,415.4
6,2021-06-25,Carla,MESA,1,124.0
1,2021-01-23,Carlos,TV,5,238.0


## Mostrando as versões agora, depois do insert

In [0]:
%sql
DESCRIBE HISTORY '/user/hive/warehouse/compras'

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
4,2023-07-25T20:20:19.000+0000,3749819615057738,andersonfaro@gmail.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(2540306768249945),0725-180139-s9tz1nlq,3,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 1528)",null,Databricks-Runtime/10.4.x-scala2.12
3,2023-07-25T20:18:36.000+0000,3749819615057738,andersonfaro@gmail.com,DELETE,"Map(predicate -> [""(cast(id#20454 as int) = 1)""])",null,List(851213765213230),0725-180139-s9tz1nlq,2,WriteSerializable,false,"Map(numRemovedFiles -> 1, numCopiedRows -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 1774, numDeletedRows -> 1, scanTimeMs -> 1211, numAddedFiles -> 0, rewriteTimeMs -> 563)",null,Databricks-Runtime/10.4.x-scala2.12
2,2023-07-25T20:18:28.000+0000,3749819615057738,andersonfaro@gmail.com,UPDATE,"Map(predicate -> [""(cast(id#19769 as int) = 4)""])",null,List(851213765213230),0725-180139-s9tz1nlq,1,WriteSerializable,false,"Map(numRemovedFiles -> 1, numCopiedRows -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 2955, scanTimeMs -> 32, numAddedFiles -> 1, numUpdatedRows -> 1, rewriteTimeMs -> 2923)",null,Databricks-Runtime/10.4.x-scala2.12
1,2023-07-25T20:18:21.000+0000,3749819615057738,andersonfaro@gmail.com,MERGE,"Map(predicate -> [""(cast(id#18912 as bigint) = id#18330L)""], matchedPredicates -> [{""actionType"":""update""}], notMatchedPredicates -> [{""actionType"":""insert""}])",null,List(851213765213230),0725-180139-s9tz1nlq,0,WriteSerializable,false,"Map(numTargetRowsCopied -> 0, numTargetRowsDeleted -> 0, numTargetFilesAdded -> 7, executionTimeMs -> 3126, numTargetRowsInserted -> 7, scanTimeMs -> 1066, numTargetRowsUpdated -> 0, numOutputRows -> 7, numTargetChangeFilesAdded -> 0, numSourceRows -> 7, numTargetFilesRemoved -> 0, rewriteTimeMs -> 1917)",null,Databricks-Runtime/10.4.x-scala2.12
0,2023-07-25T20:18:14.000+0000,3749819615057738,andersonfaro@gmail.com,CREATE OR REPLACE TABLE,"Map(isManaged -> true, description -> null, partitionBy -> [""date_order""], properties -> {})",null,List(851213765213230),0725-180139-s9tz1nlq,null,WriteSerializable,true,Map(),null,Databricks-Runtime/10.4.x-scala2.12


## Verificando quantos registro é a diferença da versão atual, para a versão 3

In [0]:
%sql
SELECT count(distinct ID) - (SELECT count(distinct ID) FROM compras
VERSION AS OF 3) as `Diferença de registros` FROM compras

Diferença de registros
1
